In [7]:
import pandas as pd
import numpy as np
import json
import requests
import os

In [8]:
police_office_forders = os.listdir('../data/police_office')
police_office_forders

['경찰청_전국 경찰관서현황_2022.csv']

In [16]:
df_police = pd.read_csv('../data/police_office/'+police_office_forders[0], encoding='UTF-8', index_col=0).reset_index()
df_police

,연번,전체기관명,도로명주소
0,1,경찰청 서울특별시경찰청 서울중부경찰서 을지지구대,서울특별시 중구 을지로 234
1,2,경찰청 서울특별시경찰청 서울중부경찰서 광희지구대,서울특별시 중구 퇴계로 375-1
2,3,경찰청 서울특별시경찰청 서울중부경찰서 약수지구대,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,4,경찰청 서울특별시경찰청 서울중부경찰서 신당파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)"
4,5,경찰청 서울특별시경찰청 서울중부경찰서 장충파출소,서울특별시 중구 동호로 261
...,...,...,...
2046,2047,경찰청 제주경찰청 서귀포경찰서 안덕파출소,제주특별자치도 서귀포시 안덕면 화순로 116
2047,2048,경찰청 제주경찰청 서귀포경찰서 대정파출소,제주특별자치도 서귀포시 대정읍 상모로 310
2048,2049,경찰청 제주경찰청 서귀포경찰서 중문파출소,제주특별자치도 서귀포시 천제연로 165
2049,2050,경찰청 제주경찰청 서귀포경찰서 대신파출소,제주특별자치도 서귀포시 신서귀로51번길 18


In [14]:
api_key = '79d3d40921cad65ee45ee1059c974972' # daero
# api_key = 'a6bd28c812cc2401914de6d7fcc93171' # jihwan

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['road_address']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

def get_lat_lon(df):
    df.insert(len(df.columns), 'geometry',[np.NaN for k in range(0,len(df))])
    for i in range(0, len(df)):
        try:
            addr = df.iloc[i]['도로명주소']
            x, y = addr_to_lat_lon(addr)
            point = [x, y]
        except:
            point = np.nan

        if x == 0 and y == 0:
            df['geometry'].at[i] = np.nan
        else:
            df['geometry'].at[i] = np.array(point).astype(object)
    

In [17]:
get_lat_lon(df_police)
df_police

,연번,전체기관명,도로명주소,geometry
0,1,경찰청 서울특별시경찰청 서울중부경찰서 을지지구대,서울특별시 중구 을지로 234,"[127.003943654119, 37.5664734170529]"
1,2,경찰청 서울특별시경찰청 서울중부경찰서 광희지구대,서울특별시 중구 퇴계로 375-1,"[127.013376356837, 37.5652684538307]"
2,3,경찰청 서울특별시경찰청 서울중부경찰서 약수지구대,"서울특별시 중구 동호로 5길 15, 약수지구대","[127.0123820902, 37.5521442775352]"
3,4,경찰청 서울특별시경찰청 서울중부경찰서 신당파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)","[127.01638496365, 37.5650442671787]"
4,5,경찰청 서울특별시경찰청 서울중부경찰서 장충파출소,서울특별시 중구 동호로 261,"[127.004802045236, 37.5587633776035]"
...,...,...,...,...
2046,2047,경찰청 제주경찰청 서귀포경찰서 안덕파출소,제주특별자치도 서귀포시 안덕면 화순로 116,"[126.334340187493, 33.2470397620401]"
2047,2048,경찰청 제주경찰청 서귀포경찰서 대정파출소,제주특별자치도 서귀포시 대정읍 상모로 310,"[126.256040570023, 33.2230746763465]"
2048,2049,경찰청 제주경찰청 서귀포경찰서 중문파출소,제주특별자치도 서귀포시 천제연로 165,"[126.422593961226, 33.2523875274983]"
2049,2050,경찰청 제주경찰청 서귀포경찰서 대신파출소,제주특별자치도 서귀포시 신서귀로51번길 18,"[126.509607469448, 33.2544182801794]"


In [18]:
missing = df_police[df_police['geometry'].isna()].index
df_police.drop(missing, inplace=True)
df_police.reset_index(inplace=True)
df_police.drop('index', axis=1, inplace=True)
df_police

,연번,전체기관명,도로명주소,geometry
0,1,경찰청 서울특별시경찰청 서울중부경찰서 을지지구대,서울특별시 중구 을지로 234,"[127.003943654119, 37.5664734170529]"
1,2,경찰청 서울특별시경찰청 서울중부경찰서 광희지구대,서울특별시 중구 퇴계로 375-1,"[127.013376356837, 37.5652684538307]"
2,3,경찰청 서울특별시경찰청 서울중부경찰서 약수지구대,"서울특별시 중구 동호로 5길 15, 약수지구대","[127.0123820902, 37.5521442775352]"
3,4,경찰청 서울특별시경찰청 서울중부경찰서 신당파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)","[127.01638496365, 37.5650442671787]"
4,5,경찰청 서울특별시경찰청 서울중부경찰서 장충파출소,서울특별시 중구 동호로 261,"[127.004802045236, 37.5587633776035]"
...,...,...,...,...
1973,2047,경찰청 제주경찰청 서귀포경찰서 안덕파출소,제주특별자치도 서귀포시 안덕면 화순로 116,"[126.334340187493, 33.2470397620401]"
1974,2048,경찰청 제주경찰청 서귀포경찰서 대정파출소,제주특별자치도 서귀포시 대정읍 상모로 310,"[126.256040570023, 33.2230746763465]"
1975,2049,경찰청 제주경찰청 서귀포경찰서 중문파출소,제주특별자치도 서귀포시 천제연로 165,"[126.422593961226, 33.2523875274983]"
1976,2050,경찰청 제주경찰청 서귀포경찰서 대신파출소,제주특별자치도 서귀포시 신서귀로51번길 18,"[126.509607469448, 33.2544182801794]"


In [19]:
df_police.to_csv(path_or_buf='../data/police_office/전국경찰관서현황(lat_lon).csv', encoding='CP949', index=False)